In [ ]:
import string
import nltk
nltk.download("stopwords")
nltk.download("punkt")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
!gdown --id 1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R
To: /content/2cls_spam_text_cls.csv
100% 486k/486k [00:00<00:00, 5.85MB/s]


In [ ]:
DATASET_PATH = "/content/2cls_spam_text_cls.csv"
df_main = pd.read_csv(DATASET_PATH)
df_main.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
messages = df_main["Message"].values.tolist()
labels = df_main["Category"].values.tolist()

print("---"*50)
print("Hiển thị Message")
print(messages)
print(len(messages))
print("---"*50)
print("Hiển thị Labels")
print(labels)
print(len(labels))

------------------------------------------------------------------------------------------------------------------------------------------------------
Hiển thị Message
['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though", "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv", 'Even my brother is not like to speak with me. They treat me like aids patent.', "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune", 'WINNER!! 

In [ ]:
def lowercase(text):
    return text.lower()

def punctuation_removal(text):
    translator = str.maketrans("","",string.punctuation)
    return text.translate(translator)

def tokenize(text):
    return text.split()

def remove_stopwords(tokens):
    stop_words = nltk.corpus.stopwords.words('english')
    return [token for token in tokens if token not in stop_words]

def stemming(tokens):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

def preprocess_text(text):
    text = lowercase(text)
    text = punctuation_removal(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = stemming(tokens)
    return tokens

messages = [preprocess_text(message) for message in messages]
print(messages)

[['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat'], ['ok', 'lar', 'joke', 'wif', 'u', 'oni'], ['free', 'entri', '2', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', '21st', 'may', '2005', 'text', 'fa', '87121', 'receiv', 'entri', 'questionstd', 'txt', 'ratetc', 'appli', '08452810075over18'], ['u', 'dun', 'say', 'earli', 'hor', 'u', 'c', 'alreadi', 'say'], ['nah', 'dont', 'think', 'goe', 'usf', 'live', 'around', 'though'], ['freemsg', 'hey', 'darl', '3', 'week', 'word', 'back', 'id', 'like', 'fun', 'still', 'tb', 'ok', 'xxx', 'std', 'chg', 'send', '£150', 'rcv'], ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'], ['per', 'request', 'mell', 'mell', 'oru', 'minnaminungint', 'nurungu', 'vettam', 'set', 'callertun', 'caller', 'press', '9', 'copi', 'friend', 'callertun'], ['winner', 'valu', 'network', 'custom', 'select', 'receivea', '£900', 'prize', 'reward', 'claim', 'call', '09061701461', '

In [ ]:
def create_dictionary(messages):
  return list({token for tokens in messages for token in tokens})

dictionary = create_dictionary(messages)
print(dictionary)

['btwn', 'waqt', 'mesh', 'lar', 'agalla', 'server', 'sathya', 'euro2004', 'u', 'goodmorningmi', '07008009200', 'flung', 'woo', 'all', 'clearer', 'hex', 'pre', 'yi', 'month', 'perumbavoor', 'filthyguy', 'perpetu', 'msgwe', 'nois', 'welli', 'comment', 'dinero', 'ha', 'level', 'gonna', 'uu', 'student', 'hot', 'yan', 'gain', 'increas', '£10000', '09066382422', 'auctionpunj', 'templ', 'unhappi', 'ceil', 'witout', 'newshyp', 'sub', '08712404000', 'grate', 'passport', 'wind', 'sink', 'bridgwat', 'mobilesdirect', 'shldxxxx', 'gaytextbuddycom', 'brief', 'okok', 'oru', 'lancast', '30th', '09058094565', 'instead', 'jesu', '48922', 'beer', 'agenc', 'joke', 'bob', 'enufcredeit', 'doublefaggot', 'shexi', 'fifth', 'themp', 'postcod', 'cornwal', 'wtc', 'muchxxlov', 'hermi', 'chikkuwat', 'partnership', 'dearshal', 'spl', 'hungov', 'kindli', 'insur', 'kintu', 'tonesrepli', 'smoke', 'pee', 'soso', 'xxsp', 'secur', 'endof', 'morn', 'peripher', 'mix', 'cttargg', 'jerk', '80086', 'kate', 'def', 'showr', 'of

In [ ]:
def create_features(tokens, dictionary):
  features = np.zeros(len(dictionary))
  for token in tokens:
    if token in dictionary:
      features[dictionary.index(token)] +=1
  return features

X = np.array([create_features(tokens, dictionary) for tokens in messages])
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
le = LabelEncoder()
y = le.fit_transform(labels)
print(f"Classes: {le.classes_}")
print(f"Encoded labels: {y}")

Classes: ['ham' 'spam']
Encoded labels: [0 0 1 ... 0 0 0]


In [ ]:
VAL_SIZE = 0.2
TEST_SIZE = 0.125
SEED = 0

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = VAL_SIZE, shuffle = True, random_state = SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, shuffle = True, random_state = SEED)


In [ ]:
model = MultinomialNB()
print("Start training...")
model = model.fit(X_train, y_train)
print("Training completed!")

Start training...
Training completed!


In [ ]:
def predict(text, model, dictionary, label_encoder):
  processed_text = preprocess_text(text)
  features = create_features(processed_text, dictionary)
  features = np.array(features).reshape(1, -1)
  prediction = model.predict(features)
  prediction_cls = label_encoder.inverse_transform(prediction)[0]
  return prediction_cls

test_input = "I am actually thinking a way of doing something useful"
print(predict(test_input, model, dictionary, le))

ham


## Phần mở rộng

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 49.2 MB/s eta 0:00:00


In [ ]:
!pip install flaml kagglehub pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 9.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import faiss
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
from datetime import datetime
from collections import Counter
import re
import warnings
import gdown
import kagglehub
from kagglehub import KaggleDatasetAdapter
warnings.filterwarnings('ignore')

## Data Augmentation

In [ ]:
# ============================================
# ✅ COMPLETE DATA AUGMENTATION MODULE
# ============================================
import nltk
import random
import pandas as pd

# Download required NLTK data
try:
    from nltk.corpus import wordnet
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    WORDNET_AVAILABLE = True
except:
    print("⚠️ NLTK WordNet not available, synonym replacement disabled")
    WORDNET_AVAILABLE = False

# ----- Hard Ham Phrase Groups -----
financial_phrases = [
    "I got $100 cashback yesterday", "The bank refunded me $200 already",
    "I earned $150/day last month from freelancing", "Approved for $500 loan finally",
    "Got quick $300 refund after confirmation", "The store gave me $250 cashback",
    "My account got $100 instantly after confirming", "I received instant $400 transfer today",
    "They sent me exclusive $600 grant, lol", "Netflix actually gave me 3 months free"
]

promotion_phrases = [
    "I bought one and got one free, legit deal", "Flash sale 80% off, I already ordered",
    "Exclusive deal worked for me, saved a lot", "Hot sale 2 hours ago, crazy cheap",
    "New collection free shipping, I tried it", "Best price ever for members",
    "Got special coupon, it worked!", "Reserved early and saved 20%",
    "Only 3 items left when I bought mine", "Order now, it's real not fake"
]

lottery_phrases = [
    "I actually won a $1000 voucher at the mall", "I got a free iPhone from the lucky draw",
    "Claimed my $500 Amazon voucher legit", "Won a prize, just showed my ticket",
    "Spun the wheel at the fair and got gifts", "Lucky draw worked for me today",
    "Redeemed my exclusive prize at the shop", "They reminded me to collect my reward",
    "Gift unlocked at the event, so fun", "Jackpot giveaway, real not scam"
]

scam_alert_phrases = [
    "I got unusual login alert, but it was me", "Reset my password after warning, fine now",
    "Got security update mail, confirmed it's real", "Payment failed once, updated and ok now",
    "Had to verify identity, bank confirmed legit", "Netflix on hold but paid, no issue",
    "Bank asked to confirm transaction, was me", "Warning mail yesterday, false alarm",
    "Confirmed within 24h, all safe", "Suspicious access blocked, just me traveling"
]

call_to_action_phrases = [
    "I clicked to confirm and it worked", "Replied YES, bonus legit",
    "Registered before midnight, no scam", "Tapped link, claimed reward legit",
    "Signed up today, limited seat real", "Confirmed immediately, nothing shady",
    "Acted fast, got discount legit", "Verified email, safe and done",
    "Downloaded app, free points real", "Paid within 12 hours, successful"
]

social_engineering_phrases = [
    "Mom, don't worry I sent you $500 hospital bill already", "Hi mom, phone broke but friend helped",
    "Boss asked me to buy gift cards for office, already did", "John, I transferred $300, check it",
    "Cousin stuck abroad, we sent help", "Friend lent me $200 last week, repaid",
    "Lost wallet but someone returned $150", "Urgent cash request yesterday, sorted now",
    "Helped pay fine, friend returned", "Sister's surgery done, paid $400 legit"
]

obfuscated_phrases = [
    "Clicked h3re to win fr€e gift, real promo", "Got r3fund n0w!!! 100% legit",
    "Fr33 reward worked, tried it", "C@shb@ck real, used today",
    "Won prize real, not spam", "Cl@imed b0nus myself, safe",
    "Gift order legit, no scam", "Refund approved @ bank, no issue",
    "Replied N0W got $$$ legit", "Urg3nt confirm done, real bank"
]

hard_ham_phrase_groups = [
    financial_phrases, promotion_phrases, lottery_phrases,
    scam_alert_phrases, call_to_action_phrases,
    social_engineering_phrases, obfuscated_phrases
]

# ----- Augmentation Functions -----
def generate_hard_ham(ham_texts, n=100):
    """Generate hard ham examples that look like spam"""
    if not ham_texts or n <= 0:
        return []

    hard_ham = []
    for _ in range(n):
        try:
            base = random.choice(ham_texts)
            insert_group = random.choice(hard_ham_phrase_groups)
            insert = random.choice(insert_group)

            if random.random() > 0.5:
                hard_ham.append(f"{base}, btw {insert}.")
            else:
                hard_ham.append(f"{insert}. {base}")
        except Exception as e:
            print(f"⚠️ Error generating hard ham: {e}")
            continue

    return hard_ham

def synonym_replacement(text, n=1):
    """Replace words with synonyms using WordNet"""
    if not WORDNET_AVAILABLE:
        return text

    try:
        # Handle input type safety
        if isinstance(text, list):
            text = ' '.join(str(item) for item in text)
        elif not isinstance(text, str):
            text = str(text)

        if not text or not text.strip():
            return text

        words = text.split()
        new_words = words.copy()

        # Filter words that have synonyms in WordNet
        candidates = []
        for w in words:
            try:
                if wordnet.synsets(w):
                    candidates.append(w)
            except:
                continue

        if not candidates:
            return text

        random.shuffle(candidates)
        replaced_count = 0

        for random_word in candidates:
            try:
                synonyms = wordnet.synsets(random_word)
                if synonyms:
                    synonym = synonyms[0].lemmas()[0].name().replace('_', ' ')
                    if synonym.lower() != random_word.lower():
                        new_words = [synonym if w == random_word else w for w in new_words]
                        replaced_count += 1
                if replaced_count >= n:
                    break
            except:
                continue

        return " ".join(new_words)

    except Exception as e:
        print(f"⚠️ Synonym replacement error: {e}")
        return str(text) if text else ""

def augment_dataset(messages, labels, aug_ratio=0.2, alpha=0.3):
    """
    Complete data augmentation combining Hard Ham Generation and Synonym Replacement

    Args:
        messages: List of message texts
        labels: List of corresponding labels ('ham'/'spam')
        aug_ratio: Ratio for synonym replacement (0.2 = 20%)
        alpha: Ratio for hard ham generation (0.3 = 30% of ham-spam difference)

    Returns:
        augmented_messages, augmented_labels: Lists of augmented data
    """
    augmented_messages = []
    augmented_labels = []

    # Ensure inputs are lists and strings
    if not isinstance(messages, list):
        messages = list(messages)
    if not isinstance(labels, list):
        labels = list(labels)

    # Convert all messages to strings
    clean_messages = []
    for msg in messages:
        if isinstance(msg, list):
            clean_messages.append(' '.join(str(item) for item in msg))
        else:
            clean_messages.append(str(msg))

    # Update messages with cleaned versions
    messages = clean_messages

    # Count current distribution
    ham_count = labels.count('ham')
    spam_count = labels.count('spam')

    print(f"📊 Original dataset: Ham={ham_count}, Spam={spam_count}")

    # 1. Hard Ham Generation (if ham > spam)
    if ham_count >= spam_count:
        ham_messages = [msg for msg, label in zip(messages, labels) if label == 'ham']
        n_hard_ham = int((ham_count - spam_count) * alpha)

        if n_hard_ham > 0 and ham_messages:
            print(f"🎯 Generating {n_hard_ham} hard ham examples...")
            hard_ham_generated = generate_hard_ham(ham_messages, n=n_hard_ham)

            if hard_ham_generated:
                augmented_messages.extend(hard_ham_generated)
                augmented_labels.extend(['ham'] * len(hard_ham_generated))
                print(f"✅ Generated {len(hard_ham_generated)} hard ham examples")

    # 2. Synonym Replacement
    max_aug_syn = int(len(messages) * aug_ratio)
    print(f"🎯 Attempting to generate ~{max_aug_syn} synonym replacement examples...")

    syn_count = 0
    attempts = 0
    max_attempts = len(messages) * 2  # Prevent infinite loop

    for msg, label in zip(messages, labels):
        if syn_count >= max_aug_syn or attempts >= max_attempts:
            break

        attempts += 1

        if random.random() > 0.8:  # 20% chance
            try:
                aug_msg = synonym_replacement(msg, n=1)

                # Only add if actually changed and meaningful
                if (aug_msg != msg and
                    len(aug_msg.strip()) > 0 and
                    len(aug_msg.split()) >= 2):  # At least 2 words

                    augmented_messages.append(aug_msg)
                    augmented_labels.append(label)
                    syn_count += 1

            except Exception as e:
                print(f"⚠️ Error in synonym replacement: {e}")
                continue

    print(f"✅ Generated {syn_count} synonym replacement examples")
    print(f"✅ Total augmented: {len(augmented_messages)} examples")

    return augmented_messages, augmented_labels

# ===== MAIN USAGE =====
def apply_augmentation(messages, labels, aug_ratio=0.2, alpha=0.3, apply_to_original=False):
    """
    Apply augmentation and optionally merge with original data

    Args:
        messages: Original messages
        labels: Original labels
        aug_ratio: Synonym replacement ratio
        alpha: Hard ham generation ratio
        apply_to_original: If True, extend original lists

    Returns:
        If apply_to_original=True: None (modifies original lists)
        If apply_to_original=False: (all_messages, all_labels)
    """
    print("=== Data Augmentation ===")

    # Generate augmented data
    augmented_messages, augmented_labels = augment_dataset(
        messages, labels, aug_ratio=aug_ratio, alpha=alpha
    )

    if apply_to_original:
        # Extend original lists
        original_count = len(messages)
        messages.extend(augmented_messages)
        labels.extend(augmented_labels)
        print(f"📈 Extended original dataset: {original_count} → {len(messages)} (+{len(augmented_messages)})")
        return None
    else:
        # Return combined data
        all_messages = messages + augmented_messages
        all_labels = labels + augmented_labels
        print(f"📈 Combined dataset: {len(messages)} → {len(all_messages)} (+{len(augmented_messages)})")
        return all_messages, all_labels

# ===== EXAMPLE USAGE =====
# Cách 1: Extend original lists
# apply_augmentation(messages, labels, apply_to_original=True)

# Cách 2: Get new combined lists
# all_messages, all_labels = apply_augmentation(messages, labels, apply_to_original=False)

# Cách 3: Manual control
# augmented_messages, augmented_labels = augment_dataset(messages, labels)
# messages.extend(augmented_messages)
# labels.extend(augmented_labels)

## import kaggle dataset

In [ ]:
def load_data_from_kaggle():
    """Load Vietnamese spam dataset from Kaggle"""
    df = kagglehub.load_dataset(
        KaggleDatasetAdapter.PANDAS,
        "victorhoward2/vietnamese-spam-post-in-social-network",
        "vi_dataset.csv"
    )
    print(f"Successfully loaded Kaggle dataset with {len(df)} records")
    print("First 5 records:")
    print(df.head())
    return df

## import googledrive

In [ ]:
def load_data_from_gdrive(file_id="1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R"):
    output_path = f"gdrive_dataset_{file_id}.csv"
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
    df_main = pd.read_csv(output_path)

    df_ham = pd.read_csv("/content/hard_ham_samples.csv")
    df_spam = pd.read_csv("/content/hard_spam_generated_auto.csv")

    df = pd.concat([df_main, df_ham, df_spam], ignore_index=True)

    print(f"Successfully loaded Google Drive dataset with {len(df)} records")
    print("First 5 records:")
    print(df.head())
    return df

## Prepare Data

In [ ]:
def preprocess_dataframe(df):
    """Preprocess the loaded dataframe to extract messages and labels"""
    print("Preprocessing dataframe...")
    print(f"Columns available: {list(df.columns)}")

    # Try to identify text and label columns
    text_column = None
    label_column = None

    # Common text column names
    text_candidates = ['message', 'text', 'content', 'email', 'post', 'comment', "texts_vi"]
    for col in df.columns:
        if col.lower() in text_candidates or 'text' in col.lower() or 'message' in col.lower():
            text_column = col
            break

    # Common label column names
    label_candidates = ['label', 'class', 'category', 'type']
    for col in df.columns:
        if col.lower() in label_candidates or 'label' in col.lower():
            label_column = col
            break

    # If not found, use first two columns
    if text_column is None:
        text_column = df.columns[0]
        print(f"Text column not found, using first column: {text_column}")

    if label_column is None:
        label_column = df.columns[1] if len(df.columns) > 1 else df.columns[0]
        print(f"Label column not found, using: {label_column}")

    print(f"Using text column: {text_column}")
    print(f"Using label column: {label_column}")

    # Clean text data
    df[text_column] = df[text_column].astype(str).fillna('')
    df = df[df[text_column].str.strip() != '']  # Remove empty texts

    # Clean labels - convert to ham/spam format
    df[label_column] = df[label_column].astype(str).str.lower()

    # Map various label formats to ham/spam
    label_mapping = {
        '0': 'ham', '1': 'spam',
        'ham': 'ham', 'spam': 'spam',
        'normal': 'ham', 'spam': 'spam',
        'legitimate': 'ham', 'phishing': 'spam',
        'not_spam': 'ham', 'is_spam': 'spam'
    }

    df[label_column] = df[label_column].map(label_mapping).fillna(df[label_column])

    # Show label distribution
    label_counts = df[label_column].value_counts()
    print(f"Label distribution:")
    for label, count in label_counts.items():
        print(f"  {label}: {count} samples")

    messages = df[text_column].tolist()
    labels = df[label_column].tolist()

    print(f"Processed {len(messages)} messages")
    return messages, labels

## Load Data

In [ ]:
def load_dataset(source='kaggle', file_id=None):
    if source == 'kaggle':
        df = load_data_from_kaggle()
    elif source == 'gdrive':
        if file_id is None:
            file_id = "1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R"  # Default ID
        df = load_data_from_gdrive(file_id)
    else:
        raise ValueError("Source must be 'kaggle' or 'gdrive'")

    if df is None:
        raise Exception(f"Failed to load data from {source}")

    messages, labels = preprocess_dataframe(df)
    return messages, labels

## Hàm tính IDF trên train

In [ ]:
def compute_idf(messages_train, tokenizer):
    """Tính IDF từ tập train"""
    N_train = len(messages_train)
    df_counts = Counter()

    for text in messages_train:
        tokens = set(tokenizer.tokenize(text))
        df_counts.update(tokens)

    idf = {tok: np.log(N_train / (df_counts[tok] + 1)) for tok in df_counts}
    return idf


## Embedding model

In [ ]:
model_name = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
# def get_embeddings(texts, model, tokenizer, device, batch_size=32):
#     embeddings = []

#     for i in tqdm(range(0, len(texts), batch_size), desc="Generating embeddings"):
#         batch_texts = texts[i:i+batch_size]

#         batch_texts_with_prefix = [f"passage: {text}" for text in batch_texts]
#         batch_dict = tokenizer(batch_texts_with_prefix, max_length=512, padding=True, truncation=True, return_tensors="pt")
#         batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

#         with torch.no_grad():
#             outputs = model(**batch_dict)
#             batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])
#             batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
#             embeddings.append(batch_embeddings.cpu().numpy())

#     return np.vstack(embeddings)

# Hàm tạo embeddings có TF‑IDF weighting
def get_embeddings_tfidf(texts, model, tokenizer, device, idf, batch_size=32):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding w/ TF-IDF"):
        batch = texts[i : i + batch_size]
        inputs = tokenizer(
            [f"passage: {t}" for t in batch],
            max_length=512, padding=True, truncation=True,
            return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            out = model(**inputs).last_hidden_state  # (B, L, D)

        ids   = inputs["input_ids"]
        mask  = inputs["attention_mask"]
        batch_emb = []
        for b in range(ids.size(0)):
            toks    = tokenizer.convert_ids_to_tokens(ids[b])
            vecs    = out[b]  # (L, D)
            # xây dựng weights vector tf-idf
            weights = [
                (idf.get(tok, 0.0) if m==1 else 0.0)
                for tok, m in zip(toks, mask[b].tolist())
            ]
            w = torch.tensor(weights, device=device).unsqueeze(1)  # (L,1)
            if w.sum().item() == 0:
                # nếu toàn bộ tổng weight của câu là 0, trả về vector attention mask ban đầu để tránh chia cho 0
                w = mask[b].unsqueeze(1).float()
            pooled = (vecs * w).sum(dim=0) / w.sum()
            batch_emb.append(pooled)
        batch_emb = torch.stack(batch_emb)            # (B, D)
        batch_emb = F.normalize(batch_emb, p=2, dim=1)  # unit vectors
        all_embs.append(batch_emb.cpu().numpy())
    return np.vstack(all_embs)

## Handle imblance label data

In [ ]:
def calculate_class_weights(labels):
    """Calculate class weights for handling imbalanced data"""
    label_counts = Counter(labels)
    total_samples = len(labels)
    num_classes = len(label_counts)

    class_weights = {}
    for label, count in label_counts.items():
        # Inverse frequency weighting
        class_weights[label] = total_samples / (num_classes * count)

    print("Class distribution:")
    for label, count in label_counts.items():
        print(f"  {label}: {count} samples (weight: {class_weights[label]:.3f})")

    return class_weights

## Compute saliency scores

In [ ]:
# def compute_saliency_scores(query_text, model, tokenizer, device, index, train_metadata, k=10):
#     """Compute saliency scores for explainability"""
#     tokens = tokenizer.tokenize(query_text)

#     if len(tokens) <= 1:
#         return np.array([1.0])

#     # Get original embedding and spam score
#     query_with_prefix = f"query: {query_text}"
#     batch_dict = tokenizer([query_with_prefix], max_length=512, padding=True, truncation=True, return_tensors="pt")
#     batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

#     with torch.no_grad():
#         outputs = model(**batch_dict)
#         original_embedding = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])
#         original_embedding = F.normalize(original_embedding, p=2, dim=1)
#         original_embedding = original_embedding.cpu().numpy().astype("float32")

#     original_scores, original_indices = index.search(original_embedding, k)
#     original_spam_score = sum(s for s, idx in zip(original_scores[0], original_indices[0])
#                              if train_metadata[idx]["label"] == "spam")

#     saliencies = []

#     # Compute saliency for each token
#     for i, token in enumerate(tokens):
#         token_mask = tokens.copy()
#         token_mask[i] = tokenizer.pad_token
#         masked_text = tokenizer.convert_tokens_to_string(token_mask)

#         masked_query = f"query: {masked_text}"
#         masked_batch_dict = tokenizer([masked_query], max_length=512, padding=True, truncation=True, return_tensors="pt")
#         masked_batch_dict = {k: v.to(device) for k, v in masked_batch_dict.items()}

#         with torch.no_grad():
#             outputs = model(**masked_batch_dict)
#             masked_embedding = average_pool(outputs.last_hidden_state, masked_batch_dict["attention_mask"])
#             masked_embedding = F.normalize(masked_embedding, p=2, dim=1)
#             masked_embedding = masked_embedding.cpu().numpy().astype("float32")

#         masked_scores, masked_indices = index.search(masked_embedding, k)
#         masked_spam_score = sum(s for s, idx in zip(masked_scores[0], masked_indices[0])
#                                if train_metadata[idx]["label"] == "spam")

#         saliency = original_spam_score - masked_spam_score
#         saliencies.append(saliency)

#     # Normalize saliencies
#     arr = np.array(saliencies)
#     if len(arr) > 1:
#         arr = (arr - arr.min()) / (np.ptp(arr) + 1e-12)
#     else:
#         arr = np.array([1.0])

#     return arr


# Hàm classify_with_knn giữ pooling weighting + explainability
def compute_saliency(query_text, k=5):
    # tính spam_score gốc
    _, neighbor_info = classify_with_knn_threshold(
        query_text, k=k, explain=False
    )
    # lấy query embedding đã weighting
    inp = tokenizer(
        [f"query: {query_text}"], max_length=512,
        padding=True, truncation=True, return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        out = model(**inp).last_hidden_state
    # pooling weighted
    toks = tokenizer.convert_ids_to_tokens(inp["input_ids"][0])
    mask = inp["attention_mask"][0]
    vecs = out[0]
    wts  = torch.tensor(
        [idf.get(t,0.0) if m==1 else 0.0 for t,m in zip(toks,mask.tolist())],
        device=device
    ).unsqueeze(1)
    if wts.sum()==0:
        wts = mask.unsqueeze(1).float()
    q_emb = F.normalize((vecs*wts).sum(0)/wts.sum(), p=2, dim=0).cpu().numpy().reshape(1,-1).astype("float32")

    D, I = index.search(q_emb, k)
    orig_spam = sum(s for s, idx in zip(D[0], I[0]) if train_metadata[idx]["label"]=="spam")

    saliencies = []
    tokens = tokenizer.tokenize(query_text)
    for i, _ in enumerate(tokens): #toàn bộ quá trình bên dưới chỉ là lặp qua từng token để che mờ sau đó đưa vào mô hình embedding để tính score, khác với
        m = tokens.copy() #average pooling khi mỗi token có trọng số đồng đều nhau trong tổng score của vector câu thì bây giờ ta có score của vector câu được tính dựa trên các token đã được gán trọng số tf-idf
        m[i] = tokenizer.pad_token
        masked = tokenizer.convert_tokens_to_string(m)
        inp2 = tokenizer(
            [f"query: {masked}"],
            max_length=512,
            padding=True,
            truncation=True,
            return_tensors="pt"
        ).to(device)
        with torch.no_grad():
            out2 = model(**inp2).last_hidden_state
        toks2 = tokenizer.convert_ids_to_tokens(inp2["input_ids"][0])
        mask2 = inp2["attention_mask"][0]
        vecs2 = out2[0]
        w2    = torch.tensor(
            [idf.get(t,0.0) if m2==1 else 0.0 for t,m2 in zip(toks2,mask2.tolist())],
            device=device
        ).unsqueeze(1)
        if w2.sum()==0:
            w2 = mask2.unsqueeze(1).float()
        emb2 = F.normalize((vecs2*w2).sum(0)/w2.sum(), p=2, dim=0).cpu().numpy().reshape(1,-1).astype("float32")
        Dm, Im = index.search(emb2, k)
        ms_spam = sum(s for s, idx in zip(Dm[0], Im[0]) if train_metadata[idx]["label"]=="spam")
        saliencies.append(orig_spam - ms_spam)
    arr = np.array(saliencies)
    arr = (arr - arr.min()) / (np.ptp(arr) + 1e-12)
    return tokens, arr

## Classification with KNN with weighted

In [ ]:
# def classify_with_weighted_knn(query_text, model, tokenizer, device, index, train_metadata, class_weights, k=10, alpha=0.5, explain=False):
#     """Enhanced KNN classification with custom weighting formula"""

#     # Get query embedding
#     query_with_prefix = f"query: {query_text}"
#     batch_dict = tokenizer([query_with_prefix], max_length=512, padding=True, truncation=True, return_tensors="pt")
#     batch_dict = {k: v.to(device) for k, v in batch_dict.items()}

#     with torch.no_grad():
#         outputs = model(**batch_dict)
#         query_embedding = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])
#         query_embedding = F.normalize(query_embedding, p=2, dim=1)
#         query_embedding = query_embedding.cpu().numpy().astype("float32")

#     # Get nearest neighbors
#     scores, indices = index.search(query_embedding, k)

#     # Compute saliency weight
#     if explain:
#         saliency_scores = compute_saliency_scores(query_text, model, tokenizer, device, index, train_metadata, k)
#         saliency_weight = np.mean(saliency_scores)
#         tokens = tokenizer.tokenize(query_text)
#     else:
#         # Quick saliency approximation
#         saliency_weight = compute_quick_saliency(query_text)
#         saliency_scores = None
#         tokens = None

#     # Calculate weighted votes
#     vote_scores = {"ham": 0.0, "spam": 0.0}
#     neighbor_info = []

#     for i in range(k):
#         neighbor_idx = indices[0][i]
#         similarity = float(scores[0][i])
#         neighbor_label = train_metadata[neighbor_idx]["label"]
#         neighbor_message = train_metadata[neighbor_idx]["message"]

#         # Apply custom weighting formula: w = (1-α)×similarity×class_weight + α×saliency_weight
#         weight = (1 - alpha) * similarity * class_weights[neighbor_label] + alpha * saliency_weight

#         vote_scores[neighbor_label] += weight

#         neighbor_info.append({
#             "score": similarity,
#             "weight": weight,
#             "label": neighbor_label,
#             "message": neighbor_message[:100] + "..." if len(neighbor_message) > 100 else neighbor_message
#         })

#     # Get prediction
#     predicted_label = max(vote_scores, key=vote_scores.get)

#     result = {
#         "prediction": predicted_label,
#         "vote_scores": vote_scores,
#         "neighbors": neighbor_info,
#         "saliency_weight": saliency_weight,
#         "alpha": alpha
#     }

#     if explain:
#         result["tokens"] = tokens
#         result["saliency_scores"] = saliency_scores

#     return result

In [ ]:
def classify_with_weighted_knn(query_text, model, tokenizer, device, index, train_metadata, class_weights, idf, k=10, alpha=0.5, explain=False):
    """Enhanced KNN classification with custom TF-IDF pooling + explainability"""

    # 1. Get TF-IDF weighted query embedding
    tokens_input = tokenizer([f"query: {query_text}"], max_length=512, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model(**tokens_input).last_hidden_state

    input_ids = tokens_input["input_ids"][0]
    attention_mask = tokens_input["attention_mask"][0]
    token_list = tokenizer.convert_ids_to_tokens(input_ids)

    vectors = output[0]
    weights = [
        idf.get(tok, 0.0) if m == 1 else 0.0
        for tok, m in zip(token_list, attention_mask.tolist())
    ]
    weight_tensor = torch.tensor(weights, device=device).unsqueeze(1)
    if weight_tensor.sum().item() == 0:
        weight_tensor = attention_mask.unsqueeze(1).float()

    pooled = (vectors * weight_tensor).sum(0) / weight_tensor.sum()
    query_embedding = F.normalize(pooled, p=2, dim=0).cpu().numpy().reshape(1, -1).astype("float32")

    # 2. KNN search
    scores, indices = index.search(query_embedding, k)

    # 3. Compute saliency weight
    if explain:
        tokens, saliency_scores = compute_saliency(query_text, k=k)  # compute_saliency() đã dùng TF-IDF
        saliency_weight = np.mean(saliency_scores)
    else:
        saliency_weight = compute_quick_saliency(query_text)
        tokens = None
        saliency_scores = None

    # 4. Weighted voting
    vote_scores = {"ham": 0.0, "spam": 0.0}
    neighbor_info = []

    for i in range(k):
        neighbor_idx = indices[0][i]
        similarity = float(scores[0][i])
        neighbor_label = train_metadata[neighbor_idx]["label"]
        neighbor_message = train_metadata[neighbor_idx]["message"]

        weight = (1 - alpha) * similarity * class_weights[neighbor_label] + alpha * saliency_weight

        vote_scores[neighbor_label] += weight

        neighbor_info.append({
            "score": similarity,
            "weight": weight,
            "label": neighbor_label,
            "message": neighbor_message[:100] + "..." if len(neighbor_message) > 100 else neighbor_message
        })

    predicted_label = max(vote_scores, key=vote_scores.get)

    result = {
        "prediction": predicted_label,
        "vote_scores": vote_scores,
        "neighbors": neighbor_info,
        "saliency_weight": saliency_weight,
        "alpha": alpha
    }

    if explain:
        result["tokens"] = tokens
        result["saliency_scores"] = saliency_scores

    return result


In [ ]:
def compute_quick_saliency(text):
    """Enhanced saliency computation for subtle spam detection"""
    words = text.lower().split()
    text_lower = text.lower()

    basic_spam_keywords = [
        'free', 'click', 'urgent', 'limited', 'offer', 'discount', 'sale', 'win', 'prize',
        'money', 'cash', 'earn', 'guaranteed', 'act now', 'call now', 'congratulations',
        'miễn phí', 'khuyến mãi', 'giảm giá', 'ưu đãi', 'thắng', 'giải thưởng', 'tiền',
        'kiếm tiền', 'đảm bảo', 'hành động ngay', 'chúc mừng', 'cơ hội', 'quà tặng'
    ]

    social_engineering_keywords = [
        'mom', 'boss', 'hr', 'manager', 'security update', 'unusual login',
        'hospital bill', 'emergency', 'help buy', 'reimburse', 'gift cards',
        'short-staffed', 'extra shifts', 'card was declined', 'warranty',
        'mẹ', 'sếp', 'nhân sự', 'cập nhật bảo mật', 'đăng nhập bất thường',
        'viện phí', 'khẩn cấp', 'giúp mua', 'hoàn tiền'
    ]

    urgency_patterns = [
        'today', 'tomorrow', 'this week', 'before friday', 'reply yes',
        'cancel anytime', 'confirm before', 'register early', 'already got mine',
        'hôm nay', 'ngày mai', 'tuần này', 'trước thứ sáu', 'trả lời có'
    ]

    money_patterns = [
        r'\$\d+', r'\d+\$', r'\d+\s*dollar', r'\d+\s*usd',
        r'\d+\s*triệu', r'\d+\s*nghìn', r'\d+\s*đồng'
    ]

    suspicious_contexts = [
        'just signed up', 'they refund', 'i already got', 'you should try',
        'can you help', 'reply if', 'book a slot', 'free diagnostics',
        'vừa đăng ký', 'họ hoàn tiền', 'tôi đã nhận', 'bạn nên thử'
    ]

    # Calculate scores
    basic_spam_score = sum(1 for word in words if any(keyword in word for keyword in basic_spam_keywords))
    social_eng_score = sum(2 for keyword in social_engineering_keywords if keyword in text_lower)  # Higher weight
    urgency_score = sum(1.5 for pattern in urgency_patterns if pattern in text_lower)

    # Money pattern detection (regex)
    import re
    money_score = 0
    for pattern in money_patterns:
        if re.search(pattern, text_lower):
            money_score += 2  # High weight for money mentions

    suspicious_score = sum(1.5 for context in suspicious_contexts if context in text_lower)

    length_factor = 1.0
    if len(words) < 5:  # Very short
        length_factor = 0.8
    elif len(words) > 50:  # Very long
        length_factor = 1.2

    # Combined saliency score
    total_score = (basic_spam_score + social_eng_score + urgency_score + money_score + suspicious_score) * length_factor

    # Normalize by text length but with minimum threshold
    saliency = min(1.0, max(0.1, total_score / max(len(words), 1) + 0.2))

    return saliency

## Find the best alpha in weight

In [ ]:
def optimize_alpha_parameter(test_embeddings, test_labels, test_metadata, index, train_metadata, class_weights, idf, k=10):
    """Find optimal alpha value for best accuracy"""
    print("Optimizing alpha parameter...")

    alpha_values = np.arange(0.0, 1.1, 0.1)
    best_alpha = 0.0
    best_accuracy = 0.0
    alpha_results = []

    for alpha in tqdm(alpha_values, desc="Testing alpha values"):
        correct = 0
        total = len(test_embeddings)

        for i in range(total):
            query_embedding = test_embeddings[i:i+1].astype("float32")
            true_label = test_metadata[i]["label"]
            query_text = test_metadata[i]["message"]

            # Use weighted classification
            result = classify_with_weighted_knn(
                query_text, model, tokenizer, device,index, train_metadata,
                class_weights,idf, k=k, alpha=alpha, explain=False
            )

            if result["prediction"] == true_label:
                correct += 1

        accuracy = correct / total
        alpha_results.append((alpha, accuracy))

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_alpha = alpha

        print(f"Alpha: {alpha:.1f}, Accuracy: {accuracy:.4f}")

    print(f"\nBest alpha: {best_alpha:.1f} with accuracy: {best_accuracy:.4f}")
    return best_alpha, alpha_results

## Classify sub-category

In [ ]:
def classify_spam_subcategory(spam_texts, model, tokenizer, device,idf):
    """Combine BERT embeddings with keyword matching"""
    if not spam_texts:
        return []

    # 1. BERT embeddings
    spam_embeddings = get_embeddings_tfidf(spam_texts, model, tokenizer, device, idf)

    # 2. Category reference embeddings
    reference_texts = {
        'spam_quangcao': "promotional discount sale offer prize win money gift free",
        'spam_hethong': "security account system alert notification verify login password"
    }

    reference_embeddings = {}
    for category, ref_text in reference_texts.items():
        ref_emb = get_embeddings_tfidf([ref_text], model, tokenizer, device, idf)[0]
        reference_embeddings[category] = ref_emb

    subcategories = []

    for i, (text, text_embedding) in enumerate(zip(spam_texts, spam_embeddings)):
        # A. BERT semantic similarity
        bert_scores = {}
        for category, ref_emb in reference_embeddings.items():
            similarity = np.dot(text_embedding, ref_emb) / (
                np.linalg.norm(text_embedding) * np.linalg.norm(ref_emb)
            )
            bert_scores[category] = similarity

        # B. Keyword matching (existing logic)
        keyword_scores = {}
        text_lower = text.lower()
        category_keywords = {
        'spam_quangcao': [
            # Vietnamese advertising keywords
            'khuyến mãi', 'giảm giá', 'sale', 'ưu đãi', 'mua ngay', 'giá rẻ', 'miễn phí',
            'quà tặng', 'voucher', 'coupon', 'giải thưởng', 'trúng thưởng', 'cơ hội', 'trúng',
            # English advertising keywords
            'discount', 'sale', 'offer', 'promotion', 'free', 'deal', 'buy now', 'limited time',
            'special offer', 'bargain', 'cheap', 'save money', 'win', 'prize', 'gift', 'won',
            'congratulations', 'claim', 'click here', '$', 'money', 'cash'
        ],
        'spam_hethong': [
            # Vietnamese system keywords
            'thông báo', 'cảnh báo', 'tài khoản', 'bảo mật', 'xác nhận', 'cập nhật',
            'hệ thống', 'đăng nhập', 'mật khẩu', 'bị khóa', 'hết hạn', 'gia hạn', 'khóa',
            # English system keywords
            'notification', 'alert', 'account', 'security', 'confirm', 'update',
            'system', 'login', 'password', 'locked', 'expired', 'renewal', 'verify',
            'suspended', 'warning', 'breach', 'urgent', 'immediately'
        ]
        }

        for category, keywords in category_keywords.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            keyword_scores[category] = score / len(keywords)  # Normalize

        # C. Combine scores (weighted)
        final_scores = {}
        for category in bert_scores.keys():
            # 70% BERT, 30% keywords
            final_scores[category] = 0.7 * bert_scores[category] + 0.3 * keyword_scores[category]

        # D. Choose best category
        if max(final_scores.values()) < 0.3:  # Low confidence
            best_category = 'spam_khac'
        else:
            best_category = max(final_scores, key=final_scores.get)

        subcategories.append(best_category)

        print(f"Text: {text[:50]}...")
        print(f"BERT scores: {bert_scores}")
        print(f"Keyword scores: {keyword_scores}")
        print(f"Final: {best_category}")

    return subcategories

In [ ]:
def classify_spam_subcategory(spam_texts, model, tokenizer, device, idf):
    """Classify spam into subcategories using BERT-based approach"""
    if not spam_texts:
        return []

    subcategories = []

    # Get embeddings for spam texts
    spam_embeddings = get_embeddings_tfidf(spam_texts, model, tokenizer, device,idf, batch_size=16)

    # Define category keywords (Vietnamese and English)
    category_keywords = {
        'spam_quangcao': [
            # Vietnamese advertising keywords
            'khuyến mãi', 'giảm giá', 'sale', 'ưu đãi', 'mua ngay', 'giá rẻ', 'miễn phí',
            'quà tặng', 'voucher', 'coupon', 'giải thưởng', 'trúng thưởng', 'cơ hội', 'trúng',
            # English advertising keywords
            'discount', 'sale', 'offer', 'promotion', 'free', 'deal', 'buy now', 'limited time',
            'special offer', 'bargain', 'cheap', 'save money', 'win', 'prize', 'gift', 'won',
            'congratulations', 'claim', 'click here', '$', 'money', 'cash'
        ],
        'spam_hethong': [
            # Vietnamese system keywords
            'thông báo', 'cảnh báo', 'tài khoản', 'bảo mật', 'xác nhận', 'cập nhật',
            'hệ thống', 'đăng nhập', 'mật khẩu', 'bị khóa', 'hết hạn', 'gia hạn', 'khóa',
            # English system keywords
            'notification', 'alert', 'account', 'security', 'confirm', 'update',
            'system', 'login', 'password', 'locked', 'expired', 'renewal', 'verify',
            'suspended', 'warning', 'breach', 'urgent', 'immediately'
        ]
    }

    for text in spam_texts:
        text_lower = text.lower()

        # Score each category
        category_scores = {}
        for category, keywords in category_keywords.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            category_scores[category] = score

        # Classify based on highest score
        if max(category_scores.values()) == 0:
            subcategories.append('spam_khac')
        else:
            best_category = max(category_scores, key=category_scores.get)
            subcategories.append(best_category)

    return subcategories

## Đánh giá model

In [ ]:
def evaluate_weighted_knn_accuracy(test_embeddings, test_labels, test_metadata, index, train_metadata, class_weights, idf, alpha, k_values=[1, 3, 5]):
    """Evaluate accuracy using weighted KNN classification"""
    results = {}
    all_errors = {}

    for k in k_values:
        print(f"\nEvaluating with k={k}, alpha={alpha:.1f}")
        correct = 0
        total = len(test_embeddings)
        errors = []

        for i in tqdm(range(total), desc=f"Evaluating k={k}"):
            query_text = test_metadata[i]["message"]
            true_label = test_metadata[i]["label"]

            # Use weighted classification
            result = classify_with_weighted_knn(
                query_text, model, tokenizer, device, index, train_metadata,
                class_weights,idf, k=k, alpha=alpha, explain=False
            )

            predicted_label = result["prediction"]

            if predicted_label == true_label:
                correct += 1
            else:
                error_info = {
                    "index": i,
                    "original_index": test_metadata[i]["index"],
                    "message": query_text,
                    "true_label": true_label,
                    "predicted_label": predicted_label,
                    "vote_scores": result["vote_scores"],
                    "neighbors": result["neighbors"]
                }
                errors.append(error_info)

        accuracy = correct / total
        results[k] = accuracy
        all_errors[k] = errors

        print(f"Accuracy with k={k}: {accuracy:.4f} ({accuracy*100:.2f}%)")
        print(f"Errors: {len(errors)}/{total}")

    return results, all_errors

In [ ]:
def enhanced_spam_classifier_pipeline(user_input, index, train_metadata, class_weights,idf, best_alpha, k=5, explain=False):
    """Enhanced spam classification with custom weighting and subcategorization"""

    print(f'\n***Classifying: "{user_input}"')
    print(f"***Using alpha={best_alpha:.1f}, k={k}")

    # Get prediction with weighted KNN
    result = classify_with_weighted_knn(
        user_input, model, tokenizer, device, index, train_metadata,
        class_weights,idf, k=k, alpha=best_alpha, explain=explain
    )

    prediction = result["prediction"]
    vote_scores = result["vote_scores"]
    neighbors = result["neighbors"]

    print(f"***Prediction: {prediction.upper()}")
    print(f"***Vote Scores: Ham={vote_scores['ham']:.3f}, Spam={vote_scores['spam']:.3f}")
    print(f"***Saliency Weight: {result['saliency_weight']:.3f}")

    # If spam, classify subcategory
    subcategory = None
    if prediction == "spam":
        subcategories = classify_spam_subcategory([user_input], model, tokenizer, device, idf)
        subcategory = subcategories[0] if subcategories else "spam_khac"
        print(f"***Spam Subcategory: {subcategory}")

    print("\n***Top neighbors:")
    for i, neighbor in enumerate(neighbors, 1):
        print(f"{i}. Label: {neighbor['label']} | Similarity: {neighbor['score']:.4f} | Weight: {neighbor['weight']:.4f}")
        print(f"   Message: {neighbor['message']}")
        print()

    final_result = {
        "prediction": prediction,
        "subcategory": subcategory,
        "vote_scores": vote_scores,
        "neighbors": neighbors,
        "saliency_weight": result["saliency_weight"],
        "alpha": best_alpha
    }

    if explain and result.get("tokens"):
        final_result["tokens"] = result["tokens"]
        final_result["saliency_scores"] = result["saliency_scores"]

    return final_result

## Run pipeline

In [ ]:
def run_enhanced_pipeline(messages, labels, test_size=0.2, use_augmentation=True):
    """Run the complete enhanced spam classification pipeline"""

    print("=== Enhanced Spam Classification Pipeline ===")

    # 1. Prepare data
    le = LabelEncoder()
    y = le.fit_transform(labels)

    # ===== THÊM DATA AUGMENTATION =====
    if use_augmentation:
        print("\n=== Data Augmentation ===")
        try:
            augmented_messages, augmented_labels = augment_dataset(messages, labels)

            if augmented_messages:  # Chỉ thêm nếu có data được generate
                # Combine original + augmented
                original_count = len(messages)
                messages = messages + augmented_messages  # Extend list
                labels = labels + augmented_labels       # Extend list

                print(f"📈 Dataset size: {original_count} → {len(messages)} (+{len(augmented_messages)})")

                # Re-encode labels with augmented data
                y = le.fit_transform(labels)
            else:
                print("ℹ️ No augmented data generated")
        except Exception as e:
            print(f"⚠️ Augmentation failed: {e}")
            print("ℹ️ Continuing with original data...")
    else:
        print("ℹ️ Data augmentation disabled")


    # 2. Tách train/test
    idxs = np.arange(len(messages))
    train_idx, test_idx = train_test_split(idxs, test_size=test_size, stratify=y, random_state=42)

    messages_train = [messages[i] for i in train_idx]
    messages_test  = [messages[i] for i in test_idx]
    y_train        = y[train_idx]
    y_test         = y[test_idx]

    # 2. Tính IDF từ messages_train
    print("📊 Computing IDF from training data...")
    idf = compute_idf(messages_train, tokenizer)


    # 3. Generate embeddings
    print("Generating embeddings...")
    # X_embeddings = get_embeddings(messages, model, tokenizer, device, idf)
    X_train_emb = get_embeddings_tfidf(messages_train, model, tokenizer, device, idf)
    X_test_emb  = get_embeddings_tfidf(messages_test, model, tokenizer, device, idf)

    # 4. Create metadata
    train_metadata = [{"index": i, "message": messages_train[i], "label": le.inverse_transform([y_train[i]])[0]} for i in range(len(messages_train))]
    test_metadata  = [{"index": i, "message": messages_test[i], "label": le.inverse_transform([y_test[i]])[0]} for i in range(len(messages_test))]


    # 5. Create FAISS index
    print("Creating FAISS index...")
    dimension = X_train_emb.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(X_train_emb.astype("float32"))


    # 6. Calculate class weights
    train_labels = [m["label"] for m in train_metadata]
    class_weights = calculate_class_weights(train_labels)

    # 7. Optimize alpha parameter
    test_labels = [m["label"] for m in test_metadata]
    best_alpha, alpha_results = optimize_alpha_parameter(
        X_test_emb, test_labels, test_metadata, index, train_metadata, class_weights, idf
    )

    # 8. Final evaluation
    print("\n=== Final Evaluation ===")
    accuracy_results, error_results = evaluate_weighted_knn_accuracy(
        X_test_emb, test_labels, test_metadata, index, train_metadata,
        class_weights,idf, best_alpha, k_values=[1, 3, 5, 7, 9, 11]
    )

    # 9. Analyze spam subcategories
    spam_texts = [m["message"] for m in test_metadata if m["label"] == "spam"]
    if spam_texts:
        print(f"\n=== Spam Subcategory Analysis ===")
        spam_subcategories = classify_spam_subcategory(spam_texts, model, tokenizer, device, idf)
        subcat_counts = Counter(spam_subcategories)

        print("Spam subcategory distribution:")
        for subcat, count in subcat_counts.items():
            print(f"  {subcat}: {count} ({count/len(spam_texts)*100:.1f}%)")

    # 10. Save results
    results = {
        "timestamp": datetime.now().isoformat(),
        "model": model_name,
        "test_size": len(X_test_emb),
        "best_alpha": best_alpha,
        "alpha_results": alpha_results,
        "accuracy_results": accuracy_results,
        "class_weights": class_weights,
        "spam_subcategories": dict(subcat_counts) if spam_texts else {}
    }

    with open("enhanced_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n*** Results saved to enhanced_results.json ***")

    return {
        "index": index,
        "train_metadata": train_metadata,
        "test_metadata": test_metadata,
        "class_weights": class_weights,
        "best_alpha": best_alpha,
        "results": results,
        "idf": idf
    }

## Apply test case

In [ ]:
def test_enhanced_classifier(pipeline_results):
    """Test the enhanced classifier with multilingual examples"""

    index = pipeline_results["index"]
    train_metadata = pipeline_results["train_metadata"]
    class_weights = pipeline_results["class_weights"]
    best_alpha = pipeline_results["best_alpha"]
    idf = pipeline_results["idf"]

    # Test examples (English and Vietnamese)
    test_examples = [
        # English spam examples
        "Congratulations! You've won $1000! Click here to claim your prize now!",
        "URGENT: Your account will be suspended. Verify immediately to avoid closure.",
        "Limited time offer! Get 50% off all products. Buy now before it's gone!",
        "Hey, just signed up for this app, they refund $150 if you register early. I already got mine, you should try.",
        "Security update: unusual login detected. Please confirm if this was you to keep your account safe.",
        "Mom, I’m fine now but I need $250 for the hospital bill. Can you transfer it today? I’ll pay you back soon.",
        "Reminder: Your subscription will renew for $29.99 tomorrow. Cancel anytime if you don’t want to continue.",
        "Hey, boss asked me to grab 3 gift cards for a client meeting. Can you help buy them first? I’ll reimburse you after the meeting.",
        "We’re short-staffed this week. Extra shifts available at $200/day, reply YES if you’re interested.",
        "Your card was declined for a $120 grocery purchase. Reply YES if this was you, NO if not.",
        "Congrats, HR is giving $50 lunch vouchers to employees who complete the quick feedback form today.",
        "Employee perk: get a $100 refund on your annual gym pass if you confirm before Friday.",
        "Hi, your laptop warranty is still active. We’re offering free diagnostics this week, reply if you want to book a slot."

        # Vietnamese spam examples
        "Chúc mừng! Bạn đã trúng 10 triệu đồng! Nhấn vào đây để nhận ngay!",
        "KHẨN CẤP: Tài khoản của bạn sẽ bị khóa. Xác nhận ngay để tránh bị đóng.",
        "Ưu đãi có thời hạn! Giảm giá 50% tất cả sản phẩm. Mua ngay kẻo hết!",

        # English ham examples
        "Hi, how are you doing? Would you like to meet for coffee tomorrow?",
        "The meeting has been rescheduled to 3 PM. Please update your calendar.",
        "Thanks for your help with the project. The presentation went very well.",

        # Vietnamese ham examples
        "Chào bạn, bạn có khỏe không? Ngày mai mình gặp nhau uống cà phê nhé?",
        "Cuộc họp đã được dời lại lúc 3 giờ chiều. Bạn vui lòng cập nhật lịch.",
        "Cảm ơn bạn đã giúp đỡ dự án. Buổi thuyết trình đã diễn ra rất tốt."
    ]

    print("\n=== Testing Enhanced Classifier ===")

    for i, example in enumerate(test_examples, 1):
        print(f"\n--- Example {i}: {example[:50]}... ---")
        result = enhanced_spam_classifier_pipeline(
            example, index, train_metadata, class_weights, idf, best_alpha, k=5, explain=False
        )

In [ ]:
def integrate_with_existing_data(messages, labels):
    """
    Integrate with your existing messages and labels data

    Args:
        messages: List of email texts
        labels: List of 'ham'/'spam' labels
    """
    print("=== Starting Enhanced Pipeline ===")

    # Run the enhanced pipeline
    pipeline_results = run_enhanced_pipeline(messages, labels)

    # Test with examples
    test_enhanced_classifier(pipeline_results)

    return pipeline_results


In [ ]:
if __name__ == "__main__":
    TEST_KAGGLE = True
    TEST_GDRIVE = True

    results_summary = {}

    # Test 1: Kaggle Dataset
    if TEST_KAGGLE:
        print("\n" + "="*60)
        print("TESTING WITH KAGGLE DATASET")
        print("="*60)


        messages, labels = load_dataset(source='kaggle')

        # Run enhanced pipeline
        pipeline_results = run_enhanced_pipeline(messages, labels, test_size=0.2, use_augmentation=True)
        results_summary['kaggle'] = {
            'samples': len(messages),
            'best_alpha': pipeline_results['best_alpha'],
            'accuracy': pipeline_results['results']['accuracy_results']
        }

        # Test with examples
        test_enhanced_classifier(pipeline_results)



    # Test 2: Google Drive Dataset
    if TEST_GDRIVE:
        print("\n" + "="*60)
        print("TESTING WITH GOOGLE DRIVE DATASET")
        print("="*60)

        messages, labels = load_dataset(source='gdrive', file_id='1N7rk-kfnDFIGMeX0ROVTjKh71gcgx-7R')

        # Run enhanced pipeline
        pipeline_results = run_enhanced_pipeline(messages, labels, test_size=0.2, use_augmentation=True)
        results_summary['gdrive'] = {
            'samples': len(messages),
            'best_alpha': pipeline_results['best_alpha'],
            'accuracy': pipeline_results['results']['accuracy_results']
        }

        # Test with examples
        test_enhanced_classifier(pipeline_results)



    # Final Summary
    print("\n" + "="*60)
    print("FINAL RESULTS SUMMARY")
    print("="*60)

    for dataset, results in results_summary.items():
        print(f"\n{dataset.upper()} Dataset:")
        if 'error' in results:
            print(f"Error: {results['error']}")
        else:
            print(f"Samples: {results['samples']}")
            print(f"Best Alpha: {results['best_alpha']:.1f}")
            print(f"Accuracy Results:")
            for k, acc in results['accuracy'].items():
                print(f"      k={k}: {acc:.4f} ({acc*100:.2f}%)")



TESTING WITH KAGGLE DATASET
Successfully loaded Kaggle dataset with 5569 records
First 5 records:
  labels                                           texts_vi
0   spam  Nhập miễn phí trong 2 WKLY Comp để giành chiến...
1    ham           Bạn không nói quá sớm ... bạn đã nói ...
2    ham  Không, tôi không nghĩ anh ấy đến USF, anh ấy s...
3    ham  Ngay cả anh trai tôi cũng không thích nói chuy...
4    ham  Theo yêu cầu của bạn 'Melle Melle (Oru Minnami...
Preprocessing dataframe...
Columns available: ['labels', 'texts_vi']
Using text column: texts_vi
Using label column: labels
Label distribution:
  ham: 4823 samples
  spam: 746 samples
Processed 5569 messages
=== Enhanced Spam Classification Pipeline ===

=== Data Augmentation ===
📊 Original dataset: Ham=4823, Spam=746
🎯 Generating 1223 hard ham examples...
✅ Generated 1223 hard ham examples
🎯 Attempting to generate ~1113 synonym replacement examples...
✅ Generated 372 synonym replacement examples
✅ Total augmented: 1595 examples
📈 Data

Embedding w/ TF-IDF: 100%|██████████| 45/45 [00:04<00:00, 10.03it/s]


Creating FAISS index...
Class distribution:
  ham: 5075 samples (weight: 0.565)
  spam: 656 samples (weight: 4.368)
Optimizing alpha parameter...


Testing alpha values:   9%|▉         | 1/11 [00:15<02:33, 15.34s/it]

Alpha: 0.0, Accuracy: 0.8967


...

***Spam Subcategory: spam_quangcao

***Top neighbors:
1. Label: spam | Similarity: 0.8344 | Weight: 0.5529
   Message: FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For...

2. Label: spam | Similarity: 0.8335 | Weight: 0.5529
   Message: FREE MESSAGE Activate your 500 FREE Text message by replying to this message with the word FREE For ...

3. Label: spam | Similarity: 0.8253 | Weight: 0.5529
   Message: Free msg. Sorry, a service you ordered from 81303 could not be delivered as you do not have sufficie...

4. Label: spam | Similarity: 0.8224 | Weight: 0.5529
   Message: 3. You have received your mobile content. Enjoy

5. Label: spam | Similarity: 0.8208 | Weight: 0.5529
   Message: I accidentally deleted the message. Resend please., btw exclusive deal just for you.


--- Example 14: KHẨN CẤP: Tài khoản của bạn sẽ bị khóa. Xác nhận n... ---

***Classifying: "KHẨN CẤP: Tài khoản của bạn sẽ bị khóa. Xác nhận ngay để tránh bị đóng."
***Us

Embedding w/ TF-IDF: 100%|██████████| 1/1 [00:00<00:00, 82.15it/s]


***Spam Subcategory: spam_hethong

***Top neighbors:
1. Label: spam | Similarity: 0.8575 | Weight: 0.3250
   Message: verify your identity to avoid account closure. Webpage s not available!

2. Label: spam | Similarity: 0.8367 | Weight: 0.3250
   Message: Sent me ur email id soon, btw bank alert: confirm transaction or account locked.

3. Label: spam | Similarity: 0.8349 | Weight: 0.3250
   Message: verify your identity to avoid account closure. Babe, iodine need your advice

4. Label: spam | Similarity: 0.8326 | Weight: 0.3250
   Message: bank alert: confirm transaction or account locked. Thanx a lot...

5. Label: spam | Similarity: 0.8325 | Weight: 0.3250
   Message: verify your identity to avoid account closure. Oh... Icic... K lor, den meet other day...


--- Example 15: Ưu đãi có thời hạn! Giảm giá 50% tất cả sản phẩm. ... ---

***Classifying: "Ưu đãi có thời hạn! Giảm giá 50% tất cả sản phẩm. Mua ngay kẻo hết!"
***Using alpha=1.0, k=5
***Prediction: SPAM
***Vote Scores: Ham=0.000

Embedding w/ TF-IDF: 100%|██████████| 1/1 [00:00<00:00, 83.55it/s]


***Spam Subcategory: spam_quangcao

***Top neighbors:
1. Label: spam | Similarity: 0.8148 | Weight: 0.2000
   Message: reserve now and get extra 20% off. So li hai... Me bored now da lecturer repeating last weeks stuff ...

2. Label: spam | Similarity: 0.8147 | Weight: 0.2000
   Message: hot sale up to 80% off. Are you this much buzy

3. Label: spam | Similarity: 0.8141 | Weight: 0.2000
   Message: hot sale up to 80% off. Haha i think i did too

4. Label: spam | Similarity: 0.8139 | Weight: 0.2000
   Message: reserve now and get extra 20% off. I'm eatin now lor, but goin back to work soon... vitamin E mounta...

5. Label: spam | Similarity: 0.8129 | Weight: 0.2000
   Message: reserve now and get extra 20% off. Are you still getting the goods.


--- Example 16: Hi, how are you doing? Would you like to meet for ... ---

***Classifying: "Hi, how are you doing? Would you like to meet for coffee tomorrow?"
***Using alpha=1.0, k=5
***Prediction: HAM
***Vote Scores: Ham=1.262, Spam=0.315
***S